# Pygame

---

Pygame je python knihovna (tedy ji musíme nainstalovat pomocí `pip install pygame` a naimportovat do našeho python souboru pomocí `import pygame`, tedy podoběn třeba jako `import random` pro náhodné generování nejen čísel). Pygame slouží pro vykreslování "okénkové" aplikace, nebudeme tedy už pracovat jen v konzoli a psát tam text, nicméně podíváme se na to, jak tvořit grafické rozhraní pro uživatele (GUI) a naučíme se, jak v tomto grafickém prostředí něco dělat, jelikož nám půjde o tvoření her, tak například ovládání hráče v prostředí. 

Začneme s tím, jak vůbec zobrazit okno:

In [ ]:
import pygame

# Inicializace Pygame
pygame.init()

# Nastavení velikosti okna (šířka, výška)
width, height = 800, 600
screen = pygame.display.set_mode((width, height))

# Titulek okna
pygame.display.set_caption("Pygame gameska")

# Herní smyčka - běží pořád dokola
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Vyplnění pozadí barvou (např. bílou)
    screen.fill((255, 255, 255))

    # Aktualizace obrazovky
    pygame.display.flip()

# Ukončení Pygame
pygame.quit()

Tento kód zobrazí bíle okno s popiskem "Pygame gameska" o velikosti 800x600 pixelů. Projděte si celý kód. Nejdřív importujeme Pygame, následovně inicializujeme (spouštíme) Pygame, následně nastavujme velikost okna, nastavujeme titulek okna a následuje herní smyčka, v této smyčce se vykresluje obrazovka (my ji vykreslujeme jen bílou), mohou se zde vypočítávát například pozice hráče a zároveň zde kontrolujeme zda uživatel neukončil hru, abychom pak mohli ukončit celou aplikaci.

Jak ale něco vykreslit? Základní geometrické útvary můžeme vykreslit pomocí `pygame.draw....`, které umístíme před příkaz `pygame.display.flip()`, který updatuje obrazovku. V Pygame funguje systém souřadnic (odpovídají pixelům obrazovky), tedy pokud chceme něco někam vykreslit, musíme uvést co a kam. Tedy např. `pygame.draw.rect(screen, (0, 0, 255), (100, 100, 200, 100))` vykreslí obdelník na obrazovku `screen` (máme ji tam jako proměnnou), s barvou v RGB `(0, 0, 255)` což je modrá, na pozici `100, 100` (střed útvaru) a s délkou a šířkou `200, 100`. Obdobně můžeme vykreslit kruh pomocí `pygame.draw.circle(screen, (255, 0, 0), (400, 300), 50)`, kde zase vykreslujeme na obrazovku `screen`, barva je červená `(255, 0, 0)`, se tředem na souřadnici `(400, 300)` a s poloměrem `50`. Nebo třeba `pygame.draw.polygon(screen, (0, 255, 0), [(200, 200), (250, 100), (300, 200)])` vykreslí zelený trojúhelník se vrcholy na souřadnících `(200, 200)`, `(250, 100)` a `(300, 200)`, pomocí `pygame.draw.polygon(...)` můžeme vykreslit i jiný mnohoúhelník, přidáme jen další souřadnici vrcholu a Pygame nám tvar spojí.

Už umíme vykreslovat tvary, ale chtělo by to i umět ovládat jejich pozici (pohybovat s nimi) pomocí WASD kláves. To můžeme udělat následovně:

In [ ]:
import pygame

# Inicializace Pygame
pygame.init()

# Nastavení velikosti okna (šířka, výška)
width, height = 800, 600
screen = pygame.display.set_mode((width, height))

# Titulek okna
pygame.display.set_caption("Pygame gameska")

# Pozice hráče - na začátku uprostřed
player_pos = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)

# Herní smyčka - běží pořád dokola
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Vyplnění pozadí barvou (např. bílou)
    screen.fill((255, 255, 255))
    
    # Zjistíme co uživatel zmáčknul
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]: # Zmáčknul W?
        player_pos.y -= 1
    if keys[pygame.K_s]: # Zmáčknul S?
        player_pos.y += 1
    if keys[pygame.K_a]: # Zmáčknul A?
        player_pos.x -= 1
    if keys[pygame.K_d]: # Zmáčknul D?
        player_pos.x += 1

    # Vykreslení kruhu na pozici player_pos
    pygame.draw.circle(screen, (255, 0, 0), player_pos, 40)

    # Aktualizace obrazovky
    pygame.display.flip()

# Ukončení Pygame
pygame.quit()


Tento kód je stejný, jako pro vykreslení prázdené obrazovky, jen jsme přidali informaci o pozici uživatele jako dvojici hodnot (`pygame.Vector2(...)`), tuhle pozici měníme na z herní smyčky, kde si zíkáme pomocí `keys = pygame.key.get_pressed()` co uživatel zmáčknul za klávesy a následně pomocí `if keys[pygame.K_s]:` atd. kontrolujeme, zda zmáčknul klávesu pro pohyb a pokud ano, tak přičítáme nebo odčítáme k souřadnicím daný posun.

Jak ale zařídit aby hráč nemohl procházet kostkami, tedy nějakými překážkami? Takto:

In [ ]:
import pygame

# Inicializace Pygame
pygame.init()

# Nastavení velikosti okna (šířka, výška)
width, height = 800, 600
screen = pygame.display.set_mode((width, height))

# Titulek okna
pygame.display.set_caption("Pygame gameska")

# Pozice hráče - na začátku uprostřed
player_pos = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)

# Čtverce přes které neprojdu
rects = [
    pygame.Rect(100, 200, 100, 100),
    pygame.Rect(500, 400, 100, 100)
]

# Herní smyčka - běží pořád dokola
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Vyplnění pozadí barvou (např. bílou)
    screen.fill((255, 255, 255))

    # Proměnná kam uložíme kam se hráč pohnul
    movement = pygame.Vector2(0, 0)
    
    # Zjistíme co uživatel zmáčknul
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        movement.y -= 1
    if keys[pygame.K_s]:
        movement.y += 1
    if keys[pygame.K_a]:
        movement.x -= 1
    if keys[pygame.K_d]:
        movement.x += 1

    # Vytvoříme hitbox pro hráče 
    player_rect = pygame.Rect(
        player_pos.x - 40,
        player_pos.y - 40,
        40 * 2,
        40 * 2
    )

    # Zkusit se pohnout v ose X
    new_rect_x = player_rect.move(movement.x, 0)
    if not any(new_rect_x.colliderect(rect) for rect in rects):
        player_pos.x += movement.x

    # Zkuse se pohnout v ose Y
    new_rect_y = player_rect.move(0, movement.y)
    if not any(new_rect_y.colliderect(rect) for rect in rects):
        player_pos.y += movement.y


    # Překážky
    for wall in rects:
        pygame.draw.rect(screen, (0, 0, 0), wall)

    # Vykreslení kruhu na pozici prayer_pos
    pygame.draw.circle(screen, (255, 0, 0), player_pos, 40)

    # Aktualizace obrazovky
    pygame.display.flip()

# Ukončení Pygame
pygame.quit()


Upravíme kód pro pohyb hráče v okně. V prvé řadě si musíme vytvořit list, kam si budeme ukládat pozice pro překážky, ten má název `rects`, následně musíme upravit i pohyb, kde nejdřív zkontrolujeme zda se na dané místo můžeme přesunout nebo ne (je tam překážka), tedy nejdřív si uložíme pohyb do proměnné `movement` a si vyrobíme nový čtverec jakožto hitbox hráče na nových pozicích, pokud nastane kolice s nějakou překážkou (kontrolujeme pomocí funkce `colliderrect(čtverec)`), tak se neposouváme dál, pokud kolize nenastane tak se tam posuneme. Následně stačí jen vykreslit překážky a samotného hráče a herní cyklus končí.  

Jak ale zlepšit grafiku hry abychom neměli jen barevné čtverce a kruhy? Stejně jako můžeme vkládat tvary do okna, tak můžeme vkládat i obrázky. V příklady níže například obrázek `player.png` na pozici hráče místo červeného kruhu:

In [ ]:
import pygame

# Inicializace Pygame
pygame.init()

# Nastavení velikosti okna
width, height = 800, 600
screen = pygame.display.set_mode((width, height))

# Titulek okna
pygame.display.set_caption("Pygame gameska")

# Načtení obrázku hráče
player_image = pygame.image.load("player.png").convert_alpha()
player_image = pygame.transform.scale(player_image, (80, 80))  # úprava velikosti obrázku

# Získání velikosti obrázku kvůli zarovnání na střed
image_width, image_height = player_image.get_size()

# Počáteční pozice hráče (uprostřed obrazovky)
player_pos = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)

# Herní smyčka
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Vyplnění pozadí barvou
    screen.fill((255, 255, 255))
    
    # Ovládání
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        player_pos.y -= 1
    if keys[pygame.K_s]:
        player_pos.y += 1
    if keys[pygame.K_a]:
        player_pos.x -= 1
    if keys[pygame.K_d]:
        player_pos.x += 1

    # Vykreslení obrázku hráče na pozici
    screen.blit(player_image, (player_pos.x - image_width / 2, player_pos.y - image_height / 2))

    # Aktualizace obrazovky
    pygame.display.flip()

# Ukončení Pygame
pygame.quit()


Kde si nejdřív načteme obrázek pomocí `pygame.image.load("player.png").convert_alpha()`, následně mu změníme velikost pomocí `pygame.transform.scale(player_image, (80, 80))`, po tomto nastavení ho můžeme vykreslovat na souřadnicích a to pomocí ` screen.blit(player_image, (player_pos.x - image_width / 2, player_pos.y - image_height / 2))`. 